<a href="https://colab.research.google.com/github/lakatosgabor/parsers/blob/main/LL1parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aritmetikai kifejezések ellenőrzése

Összeadás, kivonás, szorzás, osztás, cos, sin műveletek ellenőrzése.

In [33]:
class LL1Parser:
    def __init__(self):
        self.input_str = ""
        self.pos = 0
        self.current_token = None

    def set_input(self, input_str):
        self.input_str = input_str + '$'
        self.pos = 0
        self.current_token = self.get_next_token()

    def get_next_token(self):
        return self.input_str[self.pos]

    def lookahead(self):
        next_pos = self.pos + 1
        if next_pos < len(self.input_str):
            return self.input_str[next_pos]
        else:
            return None

    def match(self, expected_token):
        if self.current_token == expected_token:
            self.pos += 1
            self.current_token = self.get_next_token()
        else:
            raise ValueError("Syntax error")

    def E(self):
        self.T()
        self.E_prime()

    def E_prime(self):
        if self.current_token in ['+', '-']:
            op = self.current_token
            self.match(op)
            self.T()
            self.E_prime()

    def T(self):
        self.F()
        self.T_prime()

    def T_prime(self):
        if self.current_token in ['*', '/']:
            op = self.current_token
            self.match(op)
            self.F()
            self.T_prime()

    def F(self):
        if self.current_token.isdigit():
            self.N()
        elif self.current_token == '(':
            self.match('(')
            self.E()
            self.match(')')
        elif self.current_token == 's' and self.lookahead() == 'i':
            self.current_token = 'sin'
            self.pos += 2
            self.match('sin')
            self.match('(')
            self.E()
            self.match(')')
        elif self.current_token == 'c' and self.lookahead() == 'o':
            self.current_token = 'cos'
            self.pos += 2
            self.match('cos')
            self.match('(')
            self.E()
            self.match(')')
        else:
            raise ValueError("Syntax error")

    def N(self):
        if self.current_token.isdigit():
            self.match(self.current_token)
            self.N_prime()
        else:
            raise ValueError("Syntax error")

    def N_prime(self):
        if self.current_token.isdigit():
            self.match(self.current_token)
            self.N_prime()
        elif self.current_token == '.':
            self.match('.')
            self.N()

    def parse(self, input_str):
        self.set_input(input_str)
        try:
            self.E()
            if self.current_token == '$':
                print("String: ", input_str ," is accepted!")
            else:
                print("String: ", input_str ," is unexpected token!")
        except ValueError as e:
            print("String:", input_str ,"is not accepted! Error:", e)

## Tesztelés elfogadott kimenetek

In [34]:
parser = LL1Parser()
parser.parse("cos(1)")

String:  cos(1)  is accepted!


In [35]:
parser = LL1Parser()
parser.parse("2+3*(4-1)")

String:  2+3*(4-1)  is accepted!


In [36]:
parser = LL1Parser()
parser.parse("10/10-5")

String:  10/10-5  is accepted!


## Tesztelés elutasított kimenetek

In [37]:
parser = LL1Parser()
parser.parse("cos5")

String: cos5 is not accepted! Error: Syntax error


In [38]:
parser = LL1Parser()
parser.parse("5-")

String: 5- is not accepted! Error: Syntax error


In [39]:
parser = LL1Parser()
parser.parse("10%3")

String:  10%3  is unexpected token!
